In [ ]:
!pip install PyDrive

In [1]:
import os
import mykeys

project_name = 'CH13_LCEL'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = mykeys.get_key('LANG')
os.environ["LANGCHAIN_HUB_API_KEY"] = mykeys.get_key('LANG')
os.environ["OPENAI_API_KEY"] = mykeys.get_key('GPT')
os.environ["GOOGLE_API_KEY"] = mykeys.get_key('GOO')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = mykeys.get_key('HF')
os.environ["UPSTAGE_API_KEY"] = mykeys.get_key('UP')
os.environ["COHERE_API_KEY"] = mykeys.get_key('COH')
os.environ["JINA_API_KEY"] = mykeys.get_key('JINA')
os.environ["ANTHROPIC_API_KEY"] = mykeys.get_key('ANT')

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.


In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(project_name)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH13_LCEL


In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith(project_name, set_enable=False)

CH13 LangChain Expression Language(LCEL)

@chain 데코레이터로 Runnable 구성

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI


In [4]:
# 프롬프트 템플릿을 정의합니다.
prompt1 = ChatPromptTemplate.from_template("{topic} 에 대해 짧게 한글로 설명해주세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{sentence} 를 emoji를 활용한 인스타그램 게시글로 만들어주세요."
)


In [5]:
@chain
def custom_chain(text):
    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain1 = prompt1 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    output1 = chain1.invoke({"topic": text})

    # 두 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain2 = prompt2 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    # 두 번째 체인을 호출하여 파싱된 첫 번째 결과를 전달하고 최종 결과를 반환합니다.
    return chain2.invoke({"sentence": output1})


In [6]:
# custom_chain을 호출
print(custom_chain.invoke("양자역학"))


🌌✨ **양자역학의 매력!** ✨🌌

양자역학은 물리학의 한 분야로, 미시 세계에서 입자와 에너지의 행동을 설명하는 이론이에요! 🔍✨

고전 물리학과는 다르게, 양자역학에서는 입자가 특정한 위치나 속도를 가지지 않고, 🌀 확률로 설명된답니다! 또, 파동-입자 이중성을 통해 미세 입자들이 어떻게 행동하는지 이해할 수 있어요. 🌊➡️🔹

주요 개념들:
- 양자 상태 🌀
- 불확정성 원리 ❓🔄
- 양자 얽힘 🤝✨

이 이론들은 원자, 전자, 광자와 같은 미세 입자의 행동을 이해하는 데 필수적이에요. 💡🔬 그리고 양자역학은 현대 물리학, 화학, 전자공학 등 다양한 분야에 응용되고 있답니다! 🌐📈

양자역학의 세계에 함께 빠져보세요! 🚀💖 #양자역학 #물리학 #과학 #미세세계 #파동입자이중성 #양자상태 #불확정성원리 #과학의아름다움


RunnableWithMessageHistory

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 {ability} 에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요",
        ),
        # 대화 기록을 변수로 사용, history 가 MessageHistory 의 key 가 됨
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
runnable = prompt | model  # 프롬프트와 모델을 연결하여 runnable 객체 생성


In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)


In [9]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)


abc123


AIMessage(content='Cosine represents the ratio of the adjacent side to the hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 47, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nPziFq81VdfVphmYS818MYvLLfS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3aefd77b-2b9f-4ae8-8f91-e645382741c7-0', usage_metadata={'input_tokens': 47, 'output_tokens': 19, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
)


abc123


AIMessage(content='코사인은 직각삼각형에서 인접변과 빗변의 비율을 나타냅니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 91, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nQEwlV2px9xUQJqmwXZUw949Xfm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--598668bc-75bf-4dd9-8e1c-aaffdb1b20b1-0', usage_metadata={'input_tokens': 91, 'output_tokens': 40, 'total_tokens': 131, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
# 새로운 session_id로 인해 이전 대화 내용을 기억하지 않습니다.
with_message_history.invoke(
    # 수학 능력과 입력 메시지를 전달합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요"},
    # 새로운 session_id를 설정합니다.
    config={"configurable": {"session_id": "def234"}},
)


def234


AIMessage(content='수학에 능숙한 어시스턴트입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 58, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nQQQl4wRvPNWlJcuFX2haue7bLv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--76ad5d4e-7298-4392-a9ae-2981ee9e2461-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}  # 빈 딕셔너리를 초기화합니다.


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    # 주어진 user_id와 conversation_id에 해당하는 세션 기록을 반환합니다.
    if (user_id, conversation_id) not in store:
        # 해당 키가 store에 없으면 새로운 ChatMessageHistory를 생성하여 저장합니다.
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[  # 기존의 "session_id" 설정을 대체하게 됩니다.
        ConfigurableFieldSpec(
            id="user_id",  # get_session_history 함수의 첫 번째 인자로 사용됩니다.
            annotation=str,
            name="User ID",
            description="사용자의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",  # get_session_history 함수의 두 번째 인자로 사용됩니다.
            annotation=str,
            name="Conversation ID",
            description="대화의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
    ],
)


In [13]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

# chain 생성
chain = RunnableParallel({"output_message": ChatOpenAI()})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 세션 ID에 해당하는 대화 기록이 저장소에 없으면 새로운 ChatMessageHistory를 생성합니다.
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    # 세션 ID에 해당하는 대화 기록을 반환합니다.
    return store[session_id]


# 체인에 대화 기록 기능을 추가한 RunnableWithMessageHistory 객체를 생성합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    output_messages_key="output_message",
)

# 주어진 메시지와 설정으로 체인을 실행합니다.
with_message_history.invoke(
    # 혹은 "what is the definition of cosine?" 도 가능
    [HumanMessage(content="what is the definition of cosine?")],
    config={"configurable": {"session_id": "abc123"}},
)


{'output_message': AIMessage(content='Cosine is a mathematical function that, given an angle in a right-angled triangle, is equal to the ratio of the length of the side adjacent to the given angle to the length of the hypotenuse of the triangle. It is typically denoted as cos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 14, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nRSyajajZN0dypzZMJAES38PQnv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3c72babe-4b0b-4d74-b132-0dc4d9df5578-0', usage_metadata={'input_tokens': 14, 'output_tokens': 54, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_toke

In [14]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="이전의 내용을 한글로 답변해 주세요!")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)


{'output_message': AIMessage(content='코사인은 직각 삼각형에서 주어진 각에 대해 인접한 변의 길이와 빗변의 길이의 비율과 같은 수학적인 함수입니다. 일반적으로 cos로 표시됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 93, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nRflEL3mPvwPepfbrVc1iP7Rfk0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--342eb67a-80e1-4a66-b66c-1134e2b400a3-0', usage_metadata={'input_tokens': 93, 'output_tokens': 72, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}

In [15]:
with_message_history = RunnableWithMessageHistory(
    ChatOpenAI(),  # ChatOpenAI 언어 모델을 사용합니다.
    get_session_history,  # 대화 세션 기록을 가져오는 함수를 지정합니다.
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    # output_messages_key="output_message",
)


In [16]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="코사인의 의미는 무엇인가요?")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "def123"}},
)


AIMessage(content='코사인은 삼각함수 중 하나로, 한 각의 코사인 값을 구하는 함수입니다. 삼각형의 빗변과 인접한 변의 길이의 비율을 나타냅니다. 각도가 바뀔 때 코사인 값도 바뀌게 됩니다. 코사인 함수는 삼각형이나 원을 다루는 수학에서 자주 사용되며, 주로 삼각비나 삼각함수와 관련된 계산에 이용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 24, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nSLPDEkp0XdkofaHLearHGyKdOI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--dd81509e-9a1e-426e-818f-5d6ac69bb0fb-0', usage_metadata={'input_tokens': 24, 'output_tokens': 152, 'total_tokens': 176, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
from operator import itemgetter

with_message_history = RunnableWithMessageHistory(
    # "input_messages" 키를 사용하여 입력 메시지를 가져와 ChatOpenAI()에 전달합니다.
    itemgetter("input_messages") | ChatOpenAI(),
    get_session_history,  # 세션 기록을 가져오는 함수입니다.
    input_messages_key="input_messages",  # 입력 메시지의 키를 지정합니다.
)


In [18]:
with_message_history.invoke(
    {"input_messages": "코사인의 의미는 무엇인가요?"},
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "xyz123"}},
)


AIMessage(content='코사인은 삼각함수 중 하나로, 직각삼각형의 두 변의 길이를 이용하여 만들어지는 각도의 코사인 값을 나타냅니다. 간단히 말해, 코사인은 "인접변의 길이 나빗변의 길이"로 정의되며, 삼각형의 각도와 각의 변을 사용하여 계산됩니다. 코사인은 삼각형의 각도를 이용하여 변의 길이나 각의 크기를 구하는 데 사용되며, 수학적으로 많은 응용 분야에서 사용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 24, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nSeISTnX9uVlRok7GbAM8spkWMO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--da35c02a-b897-4f79-992f-711600ca49dd-0', usage_metadata={'input_tokens': 24, 'output_tokens': 190, 'total_tokens': 214, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
!pip install redis

In [ ]:
!docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest


In [20]:
# Redis 서버의 URL을 지정합니다.
REDIS_URL = "redis://localhost:6379/0"


In [21]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # 세션 ID를 기반으로 RedisChatMessageHistory 객체를 반환합니다.
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,  # 실행 가능한 객체
    get_message_history,  # 메시지 기록을 가져오는 함수
    input_messages_key="input",  # 입력 메시지의 키
    history_messages_key="history",  # 기록 메시지의 키
)


In [22]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 옵션으로 세션 ID를 "redis123" 로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)


AIMessage(content='cosine 는 삼각함수 중 하나로, 직각 삼각형에서 밑변과 빗변의 비율을 의미합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 47, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nWdGhqXP73jTTC9RzF73FASdeoF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fcbc0df2-60a8-4775-9b9a-47b3616161b9-0', usage_metadata={'input_tokens': 47, 'output_tokens': 49, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "foobar"로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)


AIMessage(content='cosine은 삼각함수 중 하나로, 직각 삼각형에서 밑변과 빗변의 비율을 의미합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 122, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nWkOvbp0oOHoIFp9VFeXtXbcFQe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--aedd2ecd-3b7f-47d6-8c38-650c0b80115e-0', usage_metadata={'input_tokens': 122, 'output_tokens': 47, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [24]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "redis456"로 지정합니다.
    config={"configurable": {"session_id": "redis456"}},
)


AIMessage(content='수학에 능하신 비서입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 60, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6nX8MfEZrT3XTLOZjGwUxoRdWOh0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--373918c4-f05f-47fb-95ca-59beebbb185b-0', usage_metadata={'input_tokens': 60, 'output_tokens': 13, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

사용자 정의 제네레이터(generator)

In [26]:
from typing import Iterator, List

from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    # 주어진 회사와 유사한 5개의 회사를 쉼표로 구분된 목록으로 작성하세요.
    "Write a comma-separated list of 5 companies similar to: {company}"
)
# 온도를 0.0으로 설정하여 ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI(temperature=0.0, model="gpt-4.1")

# 프롬프트와 모델을 연결하고 문자열 출력 파서를 적용하여 체인을 생성합니다.
str_chain = prompt | model | StrOutputParser()


In [27]:
# 데이터를 스트리밍합니다.
for chunk in str_chain.stream({"company": "Google"}):
    # 각 청크를 출력하고, 줄 바꿈 없이 버퍼를 즉시 플러시합니다.
    print(chunk, end="", flush=True)


Microsoft, Apple, Amazon, Meta, IBM

In [28]:
# 입력으로 llm 토큰의 반복자를 받아 쉼표로 구분된 문자열 리스트로 분할하는 사용자 정의 파서입니다.
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # 쉼표가 나올 때까지 부분 입력을 보관합니다.
    buffer = ""
    for chunk in input:
        # 현재 청크를 버퍼에 추가합니다.
        buffer += chunk
        # 버퍼에 쉼표가 있는 동안 반복합니다.
        while "," in buffer:
            # 버퍼를 쉼표로 분할합니다.
            comma_index = buffer.index(",")
            # 쉼표 이전의 모든 내용을 반환합니다.
            yield [buffer[:comma_index].strip()]
            # 나머지는 다음 반복을 위해 저장합니다.
            buffer = buffer[comma_index + 1 :]
    # 마지막 청크를 반환합니다.
    yield [buffer.strip()]


In [29]:
list_chain = str_chain | split_into_list  # 문자열 체인을 리스트로 분할합니다.


In [30]:
# 생성한 list_chain 이 문제없이 스트리밍되는지 확인합니다.
for chunk in list_chain.stream({"company": "Google"}):
    print(chunk, flush=True)  # 각 청크를 출력하고, 버퍼를 즉시 플러시합니다.


['Microsoft']
['Apple']
['Amazon']
['Meta']
['IBM']


In [31]:
# list_chain 에 데이터를 invoke 합니다.
list_chain.invoke({"company": "Google"})


['Microsoft', 'Apple', 'Amazon', 'Meta', 'IBM']

In [32]:
from typing import AsyncIterator


# 비동기 함수 정의
async def asplit_into_list(input: AsyncIterator[str]) -> AsyncIterator[List[str]]:
    buffer = ""
    # `input`은 `async_generator` 객체이므로 `async for`를 사용
    async for chunk in input:
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [
                buffer[:comma_index].strip()
            ]  # 쉼표를 기준으로 분할하여 리스트로 반환
            buffer = buffer[comma_index + 1:]
    yield [buffer.strip()]  # 남은 버퍼 내용을 리스트로 반환


# alist_chain 과 asplit_into_list 를 파이프라인으로 연결
alist_chain = str_chain | asplit_into_list


In [33]:
# async for 루프를 사용하여 데이터를 스트리밍합니다.
async for chunk in alist_chain.astream({"company": "Google"}):
    # 각 청크를 출력하고 버퍼를 비웁니다.
    print(chunk, flush=True)


['Microsoft']
['Apple']
['Amazon']
['Meta']
['IBM']


In [34]:
# 리스트 체인을 비동기적으로 호출합니다.
await alist_chain.ainvoke({"company": "Google"})


['Microsoft', 'Apple', 'Amazon', 'Meta', 'IBM']

Runtime Arguments 바인딩


In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 대수 기호를 사용하여 다음 방정식을 작성한 다음 풀이하세요.
            "Write out the following equation using algebraic symbols then solve it. "
            "Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        (
            "human",
            "{equation_statement}",  # 사용자가 입력한 방정식 문장을 변수로 받습니다.
        ),
    ]
)
# ChatOpenAI 모델을 초기화하고 temperature를 0으로 설정합니다.
model = ChatOpenAI(model="gpt-4.1", temperature=0)

# 방정식 문장을 입력받아 프롬프트에 전달하고, 모델에서 생성된 결과를 문자열로 파싱합니다.
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

# 예시 방정식 문장을 입력하여 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))


EQUATION:  
\( x^3 + 7 = 12 \)

SOLUTION:  
Subtract 7 from both sides:  
\( x^3 = 12 - 7 \)  
\( x^3 = 5 \)

Take the cube root of both sides:  
\( x = \sqrt[3]{5} \)

So,  
\( x = \sqrt[3]{5} \)


In [36]:
runnable = (
    # 실행 가능한 패스스루 객체를 생성하여 "equation_statement" 키에 할당합니다.
    {"equation_statement": RunnablePassthrough()}
    | prompt  # 프롬프트를 파이프라인에 추가합니다.
    | model.bind(
        stop="SOLUTION"
    )  # 모델을 바인딩하고 "SOLUTION" 토큰에서 생성을 중지하도록 설정합니다.
    | StrOutputParser()  # 문자열 출력 파서를 파이프라인에 추가합니다.
)
# "x raised to the third plus seven equals 12"라는 입력으로 파이프라인을 실행하고 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))


EQUATION:  
\( x^3 + 7 = 12 \)




In [37]:
openai_function = {
    "name": "solver",  # 함수의 이름
    # 함수의 설명: 방정식을 수립하고 해결합니다.
    "description": "Formulates and solves an equation",
    "parameters": {  # 함수의 매개변수
        "type": "object",  # 매개변수의 타입: 객체
        "properties": {  # 매개변수의 속성
            "equation": {  # 방정식 속성
                "type": "string",  # 방정식의 타입: 문자열
                "description": "The algebraic expression of the equation",  # 방정식의 대수식 표현
            },
            "solution": {  # 해답 속성
                "type": "string",  # 해답의 타입: 문자열
                "description": "The solution to the equation",  # 방정식의 해답
            },
        },
        "required": ["equation", "solution"],  # 필수 매개변수: 방정식과 해답
    },
}



In [38]:
# 다음 방정식을 대수 기호를 사용하여 작성한 다음 해결하세요.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4.1", temperature=0).bind(
    function_call={"name": "solver"},  # openai_function schema 를 바인딩합니다.
    functions=[openai_function],
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
# x의 세제곱에 7을 더하면 12와 같다
runnable.invoke("x raised to the third plus seven equals 12")


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"equation":"x^3 + 7 = 12","solution":"x^3 = 12 - 7 = 5; x = 5^(1/3)"}', 'name': 'solver'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 95, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3502f4eb73', 'id': 'chatcmpl-C6ncYrFNd6qOBDmpeZAybrRdZv3GG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--580a6dc6-f2a3-483d-ad8b-cf7637a54592-0', usage_metadata={'input_tokens': 95, 'output_tokens': 39, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [39]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # 현재 날씨를 가져오는 함수의 이름
            "description": "주어진 위치의 현재 날씨를 가져옵니다",  # 함수에 대한 설명
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시와 주, 예: San Francisco, CA",  # 위치 매개변수에 대한 설명
                    },
                    # 온도 단위 매개변수 (섭씨 또는 화씨)
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],  # 필수 매개변수 지정
            },
        },
    }
]


In [41]:
# ChatOpenAI 모델을 초기화하고 도구를 바인딩합니다.
model = ChatOpenAI(model="gpt-4.1").bind(tools=tools)
# 모델을 호출하여 샌프란시스코, 뉴욕, 로스앤젤레스의 날씨에 대해 질문합니다.
model.invoke("샌프란시스코, 뉴욕, 로스앤젤레스의 현재 날씨에 대해 알려줘?")


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kD6Qsr4eCLXiqFfLNgHn4QQ8', 'function': {'arguments': '{"location": "San Francisco, CA"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_sWMMP2PVbbnEvwpQMCPevXod', 'function': {'arguments': '{"location": "New York, NY"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_XSm8r03yuNkB4s8tBeh6WfGM', 'function': {'arguments': '{"location": "Los Angeles, CA"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 99, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-C6nczCX1kiRWE39M6f8IsovtcQl8c', 'service_tier': 'default', 'finis

폴백(fallback)

In [42]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")  # GET 요청을 생성합니다.
response = httpx.Response(
    200, request=request
)  # 200 상태 코드와 함께 응답을 생성합니다.
# "rate limit" 메시지와 응답 및 빈 본문을 포함하는 RateLimitError를 생성합니다.
error = RateLimitError("rate limit", response=response, body="")


In [46]:
# OpenAI의 ChatOpenAI 모델을 사용하여 openai_llm 객체를 생성합니다.
# max_retries를 0으로 설정하여 속도 제한 등으로 인한 재시도를 방지합니다.
openai_llm = ChatOpenAI(max_retries=0)

# Anthropic의 ChatAnthropic 모델을 사용하여 anthropic_llm 객체를 생성합니다.
anthropic_llm = ChatAnthropic(model="claude-3-7-sonnet-latest")

# openai_llm을 기본으로 사용하고, 실패 시 anthropic_llm을 대체로 사용하도록 설정합니다.
llm = openai_llm.with_fallbacks([anthropic_llm])


In [47]:
# OpenAI LLM을 먼저 사용하여 오류가 발생하는 것을 보여줍니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "닭이 길을 건넌 이유는 무엇일까요?"라는 질문을 OpenAI LLM에 전달합니다.
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        # 오류가 발생하면 오류를 출력합니다.
        print("에러 발생")


에러 발생


In [48]:
# OpenAI API 호출 시 에러가 발생하는 경우 Anthropic 으로 대체하는 코드
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "대한민국의 수도는 어디야?"라는 질문을 언어 모델에 전달하여 응답을 출력합니다.
        print(llm.invoke("대한민국의 수도는 어디야?"))
    except RateLimitError:
        # RateLimitError가 발생하면 "에러 발생"를 출력합니다.
        print("에러 발생")


content='대한민국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로, 한강을 따라 위치해 있으며 약 1,000만 명의 인구가 거주하고 있습니다.' additional_kwargs={} response_metadata={'id': 'msg_01RkfQnAV95JN64eStzQ5sEQ', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 22, 'output_tokens': 86, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'} id='run--fa7bd93e-fc18-4eb3-a6e2-400561dd368a-0' usage_metadata={'input_tokens': 22, 'output_tokens': 86, 'total_tokens': 108, 'input_token_details': {'cache_read': 0, 'cache_creation': 0, 'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}}


In [49]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "질문에 짧고 간결하게 답변해 주세요.",  # 시스템 역할 설명
        ),
        ("human", "{country} 의 수도는 어디입니까?"),  # 사용자 질문 템플릿
    ]
)
chain = prompt | llm  # 프롬프트와 언어 모델을 연결하여 체인 생성
# chain = prompt | ChatOpenAI() # 이 코드이 주석을 풀고 실행하면 "오류 발생" 문구가 출력됩니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"country": "대한민국"}))  # 체인을 호출하여 결과 출력
    except RateLimitError:  # API 비용 제한 오류 처리
        print("오류 발생")


content='대한민국의 수도는 서울입니다.' additional_kwargs={} response_metadata={'id': 'msg_01QVVeJa9NfUSXFfHmWvVsZy', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 46, 'output_tokens': 20, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'} id='run--783503ed-a00e-45f5-b2b6-773c577611e3-0' usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66, 'input_token_details': {'cache_read': 0, 'cache_creation': 0, 'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}}


In [51]:
llm = openai_llm.with_fallbacks(
    # 대체 LLM으로 anthropic_llm을 사용하고, 예외 처리할 대상으로 KeyboardInterrupt를 지정합니다.
    [anthropic_llm],
    exceptions_to_handle=(KeyboardInterrupt,),  # 예외 처리 대상을 지정합니다.
)

# 프롬프트와 LLM을 연결하여 체인을 생성합니다.
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # 체인을 호출하여 결과를 출력합니다.
        print(chain.invoke({"country": "대한민국"}))
    except RateLimitError:
        # RateLimitError 예외가 발생하면 "오류 발생"를 출력합니다.
        print("오류 발생")


오류 발생


In [52]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 프롬프트 생성
prompt_template = (
    "질문에 짧고 간결하게 답변해 주세요.\n\nQuestion:\n{question}\n\nAnswer:"
)
prompt = PromptTemplate.from_template(prompt_template)


In [53]:
# 여기서는 쉽게 오류를 발생시킬 수 있는 잘못된 모델 이름을 사용하여 체인을 생성할 것입니다.
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = prompt | chat_model


In [54]:
# fallback 체인을 생성합니다.
fallback_chain1 = prompt | ChatOpenAI(model="gpt-3.6-turbo") # 오류
fallback_chain2 = prompt | ChatOpenAI(model="gpt-3.5-turbo") # 정상
fallback_chain3 = prompt | ChatOpenAI(model="gpt-4-turbo-preview") # 정상


In [57]:
# 두 개의 체인을 결합하여 최종 체인을 생성합니다.
chain = bad_chain.with_fallbacks(
    [fallback_chain1, fallback_chain2, fallback_chain3])
# 생성된 체인을 호출하여 입력값을 전달합니다.
chain.invoke({"question": "대한민국의 수도는 어디야?"})


AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 46, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6npD5369HoKl4K1gJpslfyxyy0sr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cec6dd5-6978-4e3f-b101-aa45acf5950f-0', usage_metadata={'input_tokens': 46, 'output_tokens': 5, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})